# Pruebas de Raíz Unitaria: Verificando la Estacionariedad (ADF y KPSS) 🔬

---

## **# 1. El Concepto de Raíz Unitaria y la Necesidad de Pruebas**

En el *notebook* anterior, definimos que una serie no estacionaria a menudo contiene una **Raíz Unitaria** (ejemplo canónico: el Paseo Aleatorio, $Y_t = Y_{t-1} + \epsilon_t$).

Antes de aplicar cualquier modelo ARIMA, es obligatorio saber si la serie $Y_t$ o la serie diferenciada $Z_t$ es estacionaria. Para ello, usamos **Pruebas de Hipótesis** conocidas como Pruebas de Raíz Unitaria.

### **Consecuencias de Ignorar la No Estacionariedad**

Si modelas una serie no estacionaria, puedes caer en la **Regresión Espuria** (*Spurious Regression*).

* **Regresión Espuria:** Es una correlación estadística alta y significativa entre dos series que, en realidad, no tienen ninguna relación causal o económica, simplemente porque ambas comparten una tendencia no estacionaria. El modelo te dirá que están fuertemente relacionadas (R-cuadrado alto), pero es una ilusión estadística.

Las pruebas ADF y KPSS nos ayudan a evitar esta trampa.

---

## **# 2. Test de Dickey-Fuller Aumentado (ADF)**

El Test de Dickey-Fuller Aumentado (*Augmented Dickey-Fuller*, ADF) es la prueba de raíz unitaria más utilizada.

### **A. Las Hipótesis**

El test ADF se enfoca en determinar si un coeficiente de auto-regresión es igual a 1 (lo que indica una Raíz Unitaria).

| Hipótesis | Condición Matemática | Interpretación |
| :--- | :--- | :--- |
| **Hipótesis Nula** ($H_0$) | **Existe Raíz Unitaria.** | La serie **NO es estacionaria** (es un paseo aleatorio). |
| **Hipótesis Alternativa** ($H_A$) | **No Existe Raíz Unitaria.** | La serie **SÍ es estacionaria**. |

### **B. La Regla de Decisión**

Para la prueba ADF, queremos **Rechazar $H_0$** para concluir que la serie es estacionaria.

1.  **Observa el p-valor:** Si el p-valor es **menor que el nivel de significancia** (ej. 0.05), **Rechazamos $H_0$** y concluimos que la serie es estacionaria.
2.  **Observa el Estadístico de Prueba (ADF Statistic):** Si el valor del Estadístico de Prueba es **más negativo** que los valores críticos (ej. 1%, 5%, 10%), **Rechazamos $H_0$** y concluimos que la serie es estacionaria.

### **C. Implementación en Python**

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

# --- 1. DATOS DE EJEMPLO: Serie NO Estacionaria (con Tendencia) ---
fechas = pd.date_range('2023-01-01', periods=100, freq='M')
valores_no_estacionario = [10 + i * 1.5 + np.random.normal(0, 10) for i in range(len(fechas))]
serie_no_estacionaria = pd.Series(valores_no_estacionario, index=fechas)
# ------------------------------------------------------------------

# Ejecutar el test ADF
resultado_adf = adfuller(serie_no_estacionaria)

# Imprimir los resultados
print("Resultados del Test ADF (Serie NO Estacionaria):")
print(f"Estadístico de Prueba (ADF Stat): {resultado_adf[0]:.4f}")
print(f"p-valor: {resultado_adf[1]:.4f}")
print("Valores Críticos (Crit Value):")
for key, value in resultado_adf[4].items():
    print(f"\t{key}: {value:.4f}")

# Evaluación
if resultado_adf[1] <= 0.05:
    print("\nCONCLUSIÓN: p-valor <= 0.05. Se rechaza H0. La serie es ESTACIONARIA.")
else:
    print("\nCONCLUSIÓN: p-valor > 0.05. No se rechaza H0. La serie NO es estacionaria (contiene una Raíz Unitaria).")

## **# 3. Test de Kwiatkowski-Phillips-Schmidt-Shin (KPSS)**

El test KPSS es el complemento perfecto del ADF porque tiene **hipótesis opuestas**, ofreciendo un control de robustez en el análisis.

### **A. Las Hipótesis (Inversas al ADF)**

El test KPSS se enfoca en determinar si la serie es estacionaria alrededor de una media (o tendencia).

| Hipótesis | Condición Matemática | Interpretación |
| :--- | :--- | :--- |
| **Hipótesis Nula** ($H_0$) | La serie es Estacionaria. | La serie **NO tiene Raíz Unitaria**. |
| **Hipótesis Alternativa** ($H_A$) | La serie NO es Estacionaria. | La serie **tiene Raíz Unitaria**. |

### **B. La Regla de Decisión**

Para la prueba KPSS, queremos **NO Rechazar $H_0$** para concluir que la serie es estacionaria.

1.  **Observa el Estadístico de Prueba (KPSS Statistic):** Si el valor del Estadístico de Prueba es **menor** que los valores críticos (ej. 0.10, 0.05, 0.01), **No Rechazamos $H_0$** y concluimos que la serie es estacionaria.

2.  **Observa el p-valor:** Si el p-valor es **mayor** que el nivel de significancia (ej. 0.05), **No Rechazamos $H_0$** y concluimos que la serie es estacionaria.

### **C. Implementación en Python**

Usaremos la función `kpss` de `statsmodels`.

In [ ]:
from statsmodels.tsa.stattools import kpss

# Ejecutar el test KPSS
# regression='c' testea estacionariedad alrededor de una media constante
# regression='ct' testea estacionariedad alrededor de una tendencia (más estricto)
resultado_kpss = kpss(serie_no_estacionaria, regression='c', nlags='auto')

# Imprimir los resultados
print("\nResultados del Test KPSS (Serie NO Estacionaria):")
print(f"Estadístico de Prueba (KPSS Stat): {resultado_kpss[0]:.4f}")
# El p-valor se imprime en el segundo elemento del resultado, lo imprimimos aquí:
print(f"p-valor: {resultado_kpss[1]:.4f}") 
print("Valores Críticos (Crit Value):")
for key, value in resultado_kpss[3].items():
    print(f"\t{key}: {value:.4f}")

# Evaluación
# El valor crítico del 5% es el más usado (generalmente alrededor de 0.46)
valor_critico_5 = resultado_kpss[3]['5%']

if resultado_kpss[0] < valor_critico_5:
    print(f"\nCONCLUSIÓN: Estadístico ({resultado_kpss[0]:.4f}) < Valor Crítico 5% ({valor_critico_5:.4f}). No se rechaza H0. La serie es ESTACIONARIA.")
else:
    print(f"\nCONCLUSIÓN: Estadístico ({resultado_kpss[0]:.4f}) > Valor Crítico 5% ({valor_critico_5:.4f}). Se rechaza H0. La serie NO es estacionaria.")